In [100]:
print(f"To check the SSL certificate expiration date, we will use the OpenSSL library. This package provides a high-level interface to the functions in the OpenSSL library.")

To check the SSL certificate expiration date, we will use the OpenSSL library. This package provides a high-level interface to the functions in the OpenSSL library.


In [101]:
#!pip install pyopenssl

In [102]:
from datetime import datetime
import OpenSSL
import ssl
import os
import json
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore")

In [103]:
''' pip install python-dotenv'''
load_dotenv() # will search for .env file in local folder and load variables

True

In [104]:
source_es_host = "http://localhost:9211"
# source_es_host = os.getenv('ES_DEV_V8_HOST')

In [105]:
source_es_host = source_es_host.replace("http://","").replace("https://","")
source_es_hostname = str(source_es_host.split(':')[0])
source_es_port = str(source_es_host.split(':')[1])

In [106]:
print(source_es_hostname, source_es_port)

localhost 9211


In [107]:
response_dict = {}

In [108]:
def get_ssl_certs_hosts(source_es_hostname, source_es_port):
    ''' check if the ES cluster has any ssl certs and get the expired date '''
    try:
        cert=ssl.get_server_certificate((source_es_hostname, source_es_port))
        x509 = OpenSSL.crypto.load_certificate(OpenSSL.crypto.FILETYPE_PEM, cert)
        bytes=x509.get_notAfter()
        # print(bytes)
        timestamp = bytes.decode('utf-8')
        # print (datetime.strptime(timestamp, '%Y%m%d%H%M%S%z').date().isoformat())
        # print(datetime.strptime(timestamp, '%Y%m%d%H%M%S%z'))
        ssl_expire_date = datetime.strptime(timestamp, '%Y%m%d%H%M%S%z')
        ssl_expire_date = "{}-{}-{}".format(str(ssl_expire_date.year).zfill(2), str(ssl_expire_date.month).zfill(2), str(ssl_expire_date.day).zfill(2))
    
        response_dict.update({"ssl_certs_expire_date" : ssl_expire_date})
        response_dict.update({"ssl_certs_expire_yyyymmdd" : int(ssl_expire_date.replace("-",""))})
    except Exception as e:
        response_dict.update({"ssl_certs_expire_date" : 'no_ssl_certs'})
        response_dict.update({"ssl_certs_expire_yyyymmdd" : -1})
        print(e)
    finally:
        return response_dict

In [109]:
response_dict = get_ssl_certs_hosts(source_es_hostname, source_es_port)

[WinError 10061] No connection could be made because the target machine actively refused it


In [110]:
print(json.dumps(response_dict, indent=2)) 

{
  "ssl_certs_expire_date": "no_ssl_certs",
  "ssl_certs_expire_yyyymmdd": -1
}
